In [1]:
import findspark
findspark.init()
import pyspark
import pandas as pd
from datetime import datetime, date

# Spark SQL

In [2]:
from pyspark.sql import Row, SparkSession, SQLContext
spark = SparkSession.builder.getOrCreate()

In [3]:
# from a list of rows
df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
],  schema='a long, b double, c string, d date, e timestamp')
print(df)

# from pandas dataframe
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
print(df)

# from a RDD
rdd = spark.sparkContext.parallelize([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
])
df = spark.createDataFrame(rdd, schema=['a', 'b', 'c', 'd', 'e'])
print(df)


DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]
DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]
DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]


In [4]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [5]:
# when data volume goes up, we could change the setting to make the dataframe clearer
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2.0,string1,2000-01-01,2000-01-01 12:00:00
2,3.0,string2,2000-02-01,2000-01-02 12:00:00
3,4.0,string3,2000-03-01,2000-01-03 12:00:00


In [6]:
# wrangling with rdd
list_p = [('John',19),('Smith',29),('Adam',35),('Henry',50)]
rdd = spark.sparkContext.parallelize(list_p)
ppl = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
DF_ppl = spark.createDataFrame(ppl)

In [46]:
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/guru99-edu/R-Programming/master/adult_data.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("adult_data.csv"), header=True, inferSchema= True)
df.show(5, truncate=False)

+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|x  |age|workclass|fnlwgt|education   |educational-num|marital-status    |occupation       |relationship|race |gender|capital-gain|capital-loss|hours-per-week|native-country|income|
+---+---+---------+------+------------+---------------+------------------+-----------------+------------+-----+------+------------+------------+--------------+--------------+------+
|1  |25 |Private  |226802|11th        |7              |Never-married     |Machine-op-inspct|Own-child   |Black|Male  |0           |0           |40            |United-States |<=50K |
|2  |38 |Private  |89814 |HS-grad     |9              |Married-civ-spouse|Farming-fishing  |Husband     |White|Male  |0           |0           |50            |United-States |<=50K |
|3  |28 |Local-gov|336951|Assoc-acdm  |12             |Married-civ-spouse|Protective-serv 

In [10]:
# # Data Wrangling
# df.select('age','fnlwgt').show(5)
# df.filter(df.age > 40).count()
# df.groupby("education").count().sort("count", ascending=True).show()
# df.groupby('marital-status').agg({'capital-gain': 'mean'}).show()
df.crosstab('age', 'workclass').sort("age_workclass").show()

+-------------+---+-----------+---------+------------+-------+------------+----------------+---------+-----------+
|age_workclass|  ?|Federal-gov|Local-gov|Never-worked|Private|Self-emp-inc|Self-emp-not-inc|State-gov|Without-pay|
+-------------+---+-----------+---------+------------+-------+------------+----------------+---------+-----------+
|           17| 97|          2|       21|           2|    454|           8|               9|        2|          0|
|           18|154|          5|       15|           4|    638|          12|              20|       14|          0|
|           19|183|          6|       18|           0|    784|           6|              24|       29|          3|
|           20|184|         13|       20|           2|    834|          11|              16|       33|          0|
|           21|147|          4|       22|           0|    859|           4|              15|       44|          1|
|           22|128|         16|       34|           0|    924|           9|     

In [11]:
# the exciting part! you can use dataframe wrangling and SQL at the same time
df.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

+--------+
|count(1)|
+--------+
|   48842|
+--------+



# Spark MLlib

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *